In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        """
        Initialize SVM model.
        :param learning_rate: Learning rate for gradient descent.
        :param lambda_param: Regularization parameter.
        :param n_iters: Number of iterations for optimization.
        """
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        """
        Train the SVM model.
        :param X: Input features (n_samples, n_features).
        :param y: Target values (-1 or 1 for binary classification).
        """
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Gradient Descent
        for i in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.weights) + self.bias) >= 1
                if condition:
                    # No penalty term
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights)
                else:
                    # Apply hinge loss penalty
                    self.weights -= self.learning_rate * (2 * self.lambda_param * self.weights - np.dot(x_i, y[idx]))
                    self.bias -= self.learning_rate * y[idx]

    def predict(self, X):
        """
        Predict using the trained SVM model.
        :param X: Input features.
        :return: Predicted labels (-1 or 1).
        """
        linear_output = np.dot(X, self.weights) + self.bias
        return np.sign(linear_output)

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Select two classes for binary classification (e.g., class 0 and class 1)
X = X[y != 2]  # Remove the third class
y = y[y != 2]
y = np.where(y == 0, -1, 1)  # Convert to -1 and 1

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the SVM model
svm = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
svm.fit(X_train, y_train)

# Predict on test data
y_pred = svm.predict(X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Visualize decision boundary (for first two features)
def plot_decision_boundary(X, y, model):
    x0, x1 = np.meshgrid(
        np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 200),
        np.linspace(X[:, 1].min() - 1, X[:, 1].max() + 1, 200),
    )
    X_grid = np.c_[x0.ravel(), x1.ravel()]
    X_grid = np.c_[X_grid, np.zeros((X_grid.shape[0], X.shape[1] - 2))]  # Add padding for extra dimensions
    y_pred = model.predict(X_grid).reshape(x0.shape)

    plt.figure(figsize=(8, 6))
    plt.contourf(x0, x1, y_pred, alpha=0.3, cmap='coolwarm')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
    plt.title("SVM Decision Boundary")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.show()



Accuracy: 100.00%
